# Global Crypto Visualizations


In [19]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import hvplot.pandas
%matplotlib inline
import panel as pn

from pathlib import Path

In [20]:
file_path = "./data/latest-quotes_data_ALL.csv"
df = pd.read_csv(file_path)
df["quote.USD.last_updated"] = pd.to_datetime(df["quote.USD.last_updated"])
df.drop(['Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21'], axis=1, inplace=True)
df.head(10)


,Unnamed: 0,name,symbol,slug,num_market_pairs,is_active,cmc_rank,last_updated,quote.USD.price,quote.USD.volume_24h,quote.USD.percent_change_1h,quote.USD.percent_change_24h,quote.USD.percent_change_7d,quote.USD.percent_change_30d,quote.USD.percent_change_60d,quote.USD.percent_change_90d,quote.USD.last_updated
0,0,Bitcoin,BTC,bitcoin,8939,1,1,2021-08-13T13:19:02.000Z,46621.913290,3.011084e+10,0.640637,4.543904,13.998488,42.511239,15.494834,-4.939644,2021-08-13 13:19:02+00:00
1,1,Litecoin,LTC,litecoin,710,1,12,2021-08-13T13:19:02.000Z,181.251205,2.190324e+09,0.107650,8.689300,24.417377,39.315065,5.977775,-42.043048,2021-08-13 13:19:02+00:00
2,2,Dogecoin,DOGE,dogecoin,379,1,7,2021-08-13T13:19:03.000Z,0.278661,3.814042e+09,0.447748,4.224259,38.744745,40.263210,-14.948840,-46.998978,2021-08-13 13:19:03+00:00
3,3,Monero,XMR,monero,161,1,29,2021-08-13T13:19:04.000Z,269.918605,2.290027e+08,0.862228,2.853090,1.658224,33.478329,2.111425,-36.360225,2021-08-13 13:19:04+00:00
4,4,Ethereum,ETH,ethereum,5609,1,2,2021-08-13T13:19:02.000Z,3239.449958,2.513839e+10,0.326163,4.497529,16.609544,63.740552,28.066210,-17.088063,2021-08-13 13:19:02+00:00
5,5,Cardano,ADA,cardano,286,1,4,2021-08-13T13:18:07.000Z,2.043207,6.630972e+09,0.139518,16.226653,48.516226,61.846313,31.735741,-9.485440,2021-08-13 13:18:07+00:00
6,6,Bitcoin,BTC,bitcoin,8939,1,1,2021-08-13T13:24:02.000Z,46649.942680,3.017188e+10,0.592743,4.309974,13.763203,42.562350,14.953379,-4.932852,2021-08-13 13:24:02+00:00
7,7,Litecoin,LTC,litecoin,710,1,12,2021-08-13T13:24:02.000Z,181.338083,2.191737e+09,-0.083943,8.377851,24.593745,39.519745,5.812005,-41.964348,2021-08-13 13:24:02+00:00
8,8,Dogecoin,DOGE,dogecoin,379,1,7,2021-08-13T13:24:03.000Z,0.278758,3.818195e+09,0.210817,3.705683,38.717177,40.447567,-15.900106,-47.174834,2021-08-13 13:24:03+00:00
9,9,Monero,XMR,monero,161,1,29,2021-08-13T13:24:04.000Z,269.923059,2.289117e+08,0.763273,2.559315,1.509764,33.196793,1.817863,-36.172778,2021-08-13 13:24:04+00:00


In [21]:
df.head(50).hvplot.line(
    y='quote.USD.price',
    x='quote.USD.last_updated',
    ylabel='Price',
    xlabel='Date',
    figsize=(700,750),
    groupby='symbol',
    title='Crypto Analyzer - Price Fluctuation',
    rot=45,
    widget_location="top",
    height=500

)




Column
    [0] Row
        [0] HSpacer()
        [1] WidgetBox
            [0] Select(margin=(20, 20, 20, 20), name='symbol', options=['BTC', 'LTC', ...], value='BTC', width=250)
        [2] HSpacer()
    [1] HoloViews(DynamicMap, widget_location='top')

In [22]:
df.head(100).hvplot.line(
    y='quote.USD.percent_change_7d',
    x='quote.USD.last_updated',
    ylabel='Percent Change',
    xlabel='Date',
    figsize=(700,750),
    groupby='symbol',
    title='Crypto Analyzer - Percent Change in 7 Days',
    rot=45,
    widget_location="top",
    height=500
)

Column
    [0] Row
        [0] HSpacer()
        [1] WidgetBox
            [0] Select(margin=(20, 20, 20, 20), name='symbol', options=['BTC', 'LTC', ...], value='BTC', width=250)
        [2] HSpacer()
    [1] HoloViews(DynamicMap, widget_location='top')

In [23]:
df.head(100).hvplot.line(
    y='quote.USD.percent_change_24h',
    x='quote.USD.last_updated',
    ylabel='Percent Change',
    xlabel='Date',
    figsize=(700,750),
    groupby='symbol',
    title='Crypto Analyzer - Percent Change in 24 Hours',
    rot=45,
    widget_location="top",
    height=500
)


Column
    [0] Row
        [0] HSpacer()
        [1] WidgetBox
            [0] Select(margin=(20, 20, 20, 20), name='symbol', options=['BTC', 'LTC', ...], value='BTC', width=250)
        [2] HSpacer()
    [1] HoloViews(DynamicMap, widget_location='top')

In [24]:
file_path = "./data/ticker-news_data_20210818.csv"
df = pd.read_csv(file_path)
format='%Y-%m-%d'
df['date'] = pd.to_datetime(df['date'],format=format)
#df["date"] = pd.to_datetime(df["date"])
#df = df.set_index(pd.DatetimeIndex(df['date']))
df


,Unnamed: 0,news_url,title,text,source_name,date,topics,sentiment,tickers,rank_score
0,0,https://www.benzinga.com/markets/cryptocurrenc...,Bitcoin Adoption and How to Buy Bitcoin in Canada,Photo by Ewan Kennedy on Unsplash It's clear b...,Benzinga,2021-08-16 18:43:19+00:00,[],Positive,['BTC'],6.25
1,1,https://cointelegraph.com/news/gosats-launches...,GoSats launches Bitcoin cashback rewards card ...,Users will be able to earn yield paid in BTC o...,Cointelegraph,2021-08-15 17:00:00+00:00,[],Positive,['BTC'],6.22
2,2,https://ambcrypto.com/canada-has-embraced-bitc...,The story of Bitcoin adoption in Canada seems ...,Canada has been one of the pioneering countrie...,AMBCrypto,2021-08-13 08:00:10+00:00,[],Neutral,['BTC'],6.01
3,3,https://cointelegraph.com/news/buying-bitcoin-...,"Buying Bitcoin is like entering a minefield, B...",The Bank of Russia expressed concerns over the...,Cointelegraph,2021-08-12 10:12:20+00:00,[],Positive,['BTC'],6.22
4,4,https://bitcoinmagazine.com/markets/federal-re...,"How Taper Talk, Repos, China And U.S. Legislat...",Discussing how potential tapering by the Feder...,Bitcoin Magazine,2021-08-12 00:00:00+00:00,[],Neutral,['BTC'],5.62
...,...,...,...,...,...,...,...,...,...,...
144,151,https://cryptoticker.io/en/nba-team-dallas-mav...,"NBA Team, Dallas Mavericks, Now Accepts Dogecoin","In a seemingly unprecedented move, it seems th...",CryptoTicker,2021-03-04 23:14:37+00:00,[],Positive,['DOGE'],5.35
145,152,https://cryptobriefing.com/dogecoin-will-be-av...,Dogecoin Will Be Available in Bitcoin ATMs Acr...,CoinFlip announced that it would add support...,Crypto Briefing,2021-03-02 13:44:48+00:00,[],Positive,['DOGE'],5.37
146,2601,https://decrypt.co/78803/bitcoin-mining-revenu...,Bitcoin Mining Revenue Up $35 Million Per Day ...,Bitcoin miners will have to properly time exch...,Decrypt,2021-08-17 22:22:10+00:00,['mining'],Negative,['BTC'],6.11
147,3060,https://decrypt.co/78803/bitcoin-mining-revenu...,Bitcoin Mining Revenue Up $35 Million Per Day ...,Bitcoin miners will have to properly time exch...,Decrypt,2021-08-17 22:22:10+00:00,['mining'],Neutral,['BTC'],6.11


In [25]:
df.head(400).hvplot.bar(
    title="Correlating News Source and Rank Score",
    y='rank_score',
    x='date',
    ylabel='Rank Score',
    xlabel='Date',
    figsize=(700,750),
    groupby='source_name',
    rot=45,
    height=500,
    widget_location="top",
)
       
    
    


Column
    [0] Row
        [0] HSpacer()
        [1] WidgetBox
            [0] Select(margin=(20, 20, 20, 20), name='source_name', options=['Benzinga', 'Cointelegrap...], value='Benzinga', width=250)
        [2] HSpacer()
    [1] HoloViews(DynamicMap, widget_location='top')

In [33]:

df.head(400).hvplot.bar(
    title="Correlating News Source and Rank Score",
    y='rank_score',
    x='date',
    ylabel='Rank Score',
    xlabel='Date',
    figsize=(700,750),
    groupby='source_name',
    rot=45,
    height=500,
    widget_location="top"
    )


Column
    [0] Row
        [0] HSpacer()
        [1] WidgetBox
            [0] Select(margin=(20, 20, 20, 20), name='source_name', options=['Benzinga', 'Cointelegrap...], value='Benzinga', width=250)
        [2] HSpacer()
    [1] HoloViews(DynamicMap, widget_location='top')

In [41]:

kind = pn.widgets.Select(name='kind', value='bar', options=['bar', 'line','scatter'])
plot = df.hvplot(x= 'date', y='rank_score',kind=kind, colorbar=False, width=600, widget_location="top")


pn.Row(pn.WidgetBox(kind), plot)


Row
    [0] WidgetBox
        [0] Select(name='kind', options=['bar', 'line', ...], value='bar')
    [1] ParamFunction(function)

In [ ]:
# # Plot rank score and date ( groupby news source).
# # How does the rank scores correlate to the news source over time? 
# news_data.dataframe.hvplot.bar( 
#     y = 'rank_score',
#     ylabel = " Rank Score ",
#     xlabel = "Date",
#     title = " Trend of Rank Score Over Time for Different News Sources ",
#     ylim=(6.0,7.0),
#     groupby='source_name'
# ) 

# # Plot a composite side-by-side or overlay plot using + or *. 

# # Plot sentiment and date.
# # How does the sentiment correlate to the new source over time?
# # Does the rank score and sentiments correlate in the same pattern (for the same news sources) ?

# news_data.dataframe.hvplot.line( 
#     y = 'sentiment',
#     ylabel = " Sentiment",
#     xlabel = "Date",
#     title = " Trend of Sentiment Over Time for Different News Sources ",
#     ylim=(6.0,7.0),
#     groupby='source_name'
# ) 

In [ ]:
# # Plot percent change & market pair ( groupby newsource).
# # Did the change in cypto price and number of trades that day correlate? 
# # Is there a trend dependening on the news source?
# news_data.dataframe.hvplot.line( 
#     y = ['num_market_pairs', 'quote.USD.percent_change_24h'],
#     ylabel = " Percent Change24h/Market Pairs",
#     xlabel = "Date",
#     title = " Percent Change and Market Pairs Based on the News Source ",
#     ylim=(6.0,7.0),
#     groupby='source_name'
# ) 